In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from collections import Counter
import random
import colorsys
import hdbscan
import umap
# bokeh basics
from bokeh.io import show, output_notebook
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper
from bokeh.palettes import plasma
from bokeh.plotting import figure
from bokeh.transform import transform
import bokeh.models as bmo
from sklearn.cluster import KMeans

def _get_colors(num_colors):
    colors=[]
    for i in np.arange(0., 360., 360. / num_colors):
        hue = i/360.
        lightness = (50 + np.random.rand() * 10)/100.
        saturation = (90 + np.random.rand() * 10)/100.
        colors.append(colorsys.hls_to_rgb(hue, lightness, saturation))
    return colors

thousand_colors = ['#%02x%02x%02x' % (int(c[0]*255), int(c[1]*255), int(c[2]*255)) for c in _get_colors(15)]
random.shuffle(thousand_colors)

/Users/nicolasparraavila/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/Users/nicolasparraavila/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
df = pd.read_excel('Areas y población Mun_COL 2005_2017.xlsx').iloc[:-1,:]

#### Adding new features

An initial thing to do is to add features that are important in measuring the expansion of a town. In particular we add new variables
\begin{align}
\rho_\text{year} &= \frac{\text{population in year}}{\text{area in year}}\\
\Delta \text{population} &= \frac{\text{population in 2017}}{ \text{population in 2005}}\\
\Delta \text{area} &= \frac{\text{area in 2017} }{ \text{area in 2005}}\\
C &= \rho_{2017}-\rho_{2005}
\end{align}

In [4]:
df_augmented=df.copy(deep=True)
df_augmented=df.copy(deep=True)

df_augmented['dens_2005_cab'] = df_augmented.iloc[:, 5].values/(df_augmented.iloc[:, 10].values + 1)
df_augmented['dens_2005_res'] = df_augmented.iloc[:, 6].values/(df_augmented.iloc[:, 12].values + 1)
df_augmented['dens_2017_cab'] = df_augmented.iloc[:, 8].values/(df_augmented.iloc[:, 13].values + 1)
df_augmented['dens_2017_res'] = df_augmented.iloc[:, 9].values/(df_augmented.iloc[:, 15].values + 1)

df_augmented['del_pop_tot'] = df_augmented.iloc[:, 7].values / (df_augmented.iloc[:, 4].values + 1)
df_augmented['del_pop_cab'] = df_augmented.iloc[:, 8].values / (df_augmented.iloc[:, 5].values + 1)
df_augmented['del_pop_res'] = df_augmented.iloc[:, 9].values / (df_augmented.iloc[:, 6].values + 1)

df_augmented['del_are_cab'] = df_augmented.iloc[:, 13].values / (df_augmented.iloc[:, 10].values + 1)
df_augmented['del_are_cen'] = df_augmented.iloc[:, 14].values / (df_augmented.iloc[:, 11].values + 1)
df_augmented['del_are_res'] = df_augmented.iloc[:, 15].values / (df_augmented.iloc[:, 12].values + 1)

df_augmented['c_cab'] = df_augmented['dens_2017_cab'].values - df_augmented['dens_2005_cab'].values
df_augmented['c_res'] = df_augmented['dens_2017_res'].values - df_augmented['dens_2005_res'].values

df_augmented=df_augmented[df_augmented['del_pop_tot']<5]
df_augmented=df_augmented[df_augmented['del_pop_cab']<5]
df_augmented=df_augmented[df_augmented['del_pop_res']<5]
df_augmented=df_augmented[df_augmented['del_are_cab']<5]
df_augmented=df_augmented[df_augmented['del_are_cen']<5]
df_augmented=df_augmented[df_augmented['del_are_res']<5]

data = np.hstack((np.log(df_augmented.iloc[:, 4:16].values + 1), df_augmented.iloc[:, 16:]) )
#scaler = StandardScaler()
#scaler.fit(data)
#augmented_scaled = scaler.transform(data)
augmented_scaled = data
reducer = umap.UMAP(n_neighbors=50, min_dist=0, n_components=10)
embedded = reducer.fit_transform(data)

/Users/nicolasparraavila/anaconda3/lib/python3.7/site-packages/umap/umap_.py:349: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "fuzzy_simplicial_set" failed type inference due to: Untyped global name 'nearest_neighbors': cannot determine Numba type of <class 'function'>

File "../../../../anaconda3/lib/python3.7/site-packages/umap/umap_.py", line 467:
def fuzzy_simplicial_set(
    <source elided>
    if knn_indices is None or knn_dists is None:
        knn_indices, knn_dists, _ = nearest_neighbors(
        ^

  @numba.jit()
/Users/nicolasparraavila/anaconda3/lib/python3.7/site-packages/numba/compiler.py:725: NumbaWarning: Function "fuzzy_simplicial_set" was compiled in object mode without forceobj=True.

File "../../../../anaconda3/lib/python3.7/site-packages/umap/umap_.py", line 350:
@numba.jit()
def fuzzy_simplicial_set(
^

  self.func_ir.loc))
/Users/nicolasparraavila/anaconda3/lib/python3.7/site-packages/numba/compiler.py:734: 

In [5]:
kmeans = KMeans(n_clusters=6, random_state=0,).fit(embedded)
reducer = umap.UMAP(n_neighbors=20, min_dist=0.9,n_components=3)
embedding = reducer.fit_transform(embedded)
#plt.scatter(*embedding.T,alpha=0.2,c=kmeans.labels_)
#plt.show()

In [6]:
from mpl_toolkits import mplot3d
%matplotlib qt

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter(*embedding.T,s=50,alpha=0.5,c=kmeans.labels_)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plt.show()

In [7]:
mask_0=kmeans.labels_==0
mask_1=kmeans.labels_==1
mask_2=kmeans.labels_==2
mask_3=kmeans.labels_==3
mask_4=kmeans.labels_==4
mask_5=kmeans.labels_==5

rho_2005_cab_0=df_augmented['dens_2005_cab'][mask_0]
rho_2005_cab_1=df_augmented['dens_2005_cab'][mask_1]
rho_2005_cab_2=df_augmented['dens_2005_cab'][mask_2]
rho_2005_cab_3=df_augmented['dens_2005_cab'][mask_3]
rho_2005_cab_4=df_augmented['dens_2005_cab'][mask_4]
rho_2005_cab_5=df_augmented['dens_2005_cab'][mask_5]

rho_2017_cab_0=df_augmented['dens_2017_cab'][mask_0]
rho_2017_cab_1=df_augmented['dens_2017_cab'][mask_1]
rho_2017_cab_2=df_augmented['dens_2017_cab'][mask_2]
rho_2017_cab_3=df_augmented['dens_2017_cab'][mask_3]
rho_2017_cab_4=df_augmented['dens_2017_cab'][mask_4]
rho_2017_cab_5=df_augmented['dens_2017_cab'][mask_5]

In [21]:
fig, axs = plt.subplots(2, 3, tight_layout=True,figsize=(13, 8))

axs[0,0].set_title('Histogram Cluster 1')
axs[0,1].set_title('Histogram Cluster 2')
axs[0,2].set_title('Histogram Cluster 3')
axs[1,0].set_title('Histogram Cluster 4')
axs[1,1].set_title('Histogram Cluster 5')
axs[1,2].set_title('Histogram Cluster 6')

axs[0,0].hist2d(rho_2005_cab_0,rho_2017_cab_0,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 1')
axs[0,1].hist2d(rho_2005_cab_1,rho_2017_cab_1,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 2')
axs[0,2].hist2d(rho_2005_cab_2,rho_2017_cab_2,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 3')
axs[1,0].hist2d(rho_2005_cab_3,rho_2017_cab_3,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 4')
axs[1,1].hist2d(rho_2005_cab_4,rho_2017_cab_4,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 5')
axs[1,2].hist2d(rho_2005_cab_5,rho_2017_cab_5,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 6')
#plt.ylabel('frecuency of delta de pobalcion')

axs[0,0].set_xlabel('Densidad de cabezera 2005')
axs[0,1].set_xlabel('Densidad de cabezera 2005')
axs[0,2].set_xlabel('Densidad de cabezera 2005')
axs[1,0].set_xlabel('Densidad de cabezera 2005')
axs[1,1].set_xlabel('Densidad de cabezera 2005')
axs[1,2].set_xlabel('Densidad de cabezera 2005')
axs[0,0].set_ylabel('Densidad de cabezera 2017')
axs[0,1].set_ylabel('Densidad de cabezera 2017')
axs[0,2].set_ylabel('Densidad de cabezera 2017')
axs[1,0].set_ylabel('Densidad de cabezera 2017')
axs[1,1].set_ylabel('Densidad de cabezera 2017')
axs[1,2].set_ylabel('Densidad de cabezera 2017')

plt.show()

In [24]:
mask_0=kmeans.labels_==0
mask_1=kmeans.labels_==1
mask_2=kmeans.labels_==2
mask_3=kmeans.labels_==3
mask_4=kmeans.labels_==4
mask_5=kmeans.labels_==5

rho_2005_res_0=df_augmented['dens_2005_res'][mask_0]
rho_2005_res_1=df_augmented['dens_2005_res'][mask_1]
rho_2005_res_2=df_augmented['dens_2005_res'][mask_2]
rho_2005_res_3=df_augmented['dens_2005_res'][mask_3]
rho_2005_res_4=df_augmented['dens_2005_res'][mask_4]
rho_2005_res_5=df_augmented['dens_2005_res'][mask_5]

rho_2017_res_0=df_augmented['dens_2017_res'][mask_0]
rho_2017_res_1=df_augmented['dens_2017_res'][mask_1]
rho_2017_res_2=df_augmented['dens_2017_res'][mask_2]
rho_2017_res_3=df_augmented['dens_2017_res'][mask_3]
rho_2017_res_4=df_augmented['dens_2017_res'][mask_4]
rho_2017_res_5=df_augmented['dens_2017_res'][mask_5]

In [26]:
fig, axs = plt.subplots(2, 3, tight_layout=True,figsize=(13, 8))

axs[0,0].set_title('Histogram Cluster 1')
axs[0,1].set_title('Histogram Cluster 2')
axs[0,2].set_title('Histogram Cluster 3')
axs[1,0].set_title('Histogram Cluster 4')
axs[1,1].set_title('Histogram Cluster 5')
axs[1,2].set_title('Histogram Cluster 6')

axs[0,0].hist2d(rho_2005_res_0,rho_2017_res_0,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 1')
axs[0,1].hist2d(rho_2005_res_1,rho_2017_res_1,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 2')
axs[0,2].hist2d(rho_2005_res_2,rho_2017_res_2,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 3')
axs[1,0].hist2d(rho_2005_res_3,rho_2017_res_3,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 4')
axs[1,1].hist2d(rho_2005_res_4,rho_2017_res_4,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 5')
axs[1,2].hist2d(rho_2005_res_5,rho_2017_res_5,bins=40, alpha=1, edgecolor = 'black',  linewidth=1,label='Cluster 6')
#plt.ylabel('frecuency of delta de pobalcion')

axs[0,0].set_xlabel('Densidad de resto 2005')
axs[0,1].set_xlabel('Densidad de resto 2005')
axs[0,2].set_xlabel('Densidad de resto 2005')
axs[1,0].set_xlabel('Densidad de resto 2005')
axs[1,1].set_xlabel('Densidad de resto 2005')
axs[1,2].set_xlabel('Densidad de resto 2005')
axs[0,0].set_ylabel('Densidad de resto 2017')
axs[0,1].set_ylabel('Densidad de resto 2017')
axs[0,2].set_ylabel('Densidad de resto 2017')
axs[1,0].set_ylabel('Densidad de resto 2017')
axs[1,1].set_ylabel('Densidad de resto 2017')
axs[1,2].set_ylabel('Densidad de resto 2017')


plt.show()